## Funktionen

> Potenzielles TODO: Zu jeder hier definierten Funktion schreiben in welchem Notebook sie verwendet werden.

Neben den Importen werden auch drei Funktionen definiert, die über die verwendeten Notebooks hinweg benötigt werden. Für die Verwendung muss ebenfalls dieses Notebook zuerst ausgeführt werden.

> %run Util/01_functions.ipynb

Die erste Funktion, die innerhalb dieses Notebooks definiert wird, befasst sich mit der Umwandlung von einer der drei genutzten Darstellungsweisen in eine andere. Bei diesen Darstellungsweisen handelt es sich um:

> TODO: Für jede Darstellungsweise ein Beispiel aufführen
> TODO: Darstellungsweisen updaten
> TODO: Darstellung in ganzen Sätzen beschreiben

* Die Forsyth-Edward-Notation (FEN). Diese besteht aus insgesamt sechs Teilstrings, die zur Beschreibung einer Schachpartie benötigt werden. Eine FEN besteht aus folgenden Elementen:
    * Die Platzierung der Spielfiguren auf dem Schachbrett.
    * Die Farbe, die gerade am Zug ist.
    * Die Möglichkeiten, noch eine Rochade durchzuführen.
    * Die Felder hinter einem Bauern, der per En Passant geschlagen werden können.
    * Die Anzahl an Halbzügen, seitdem die letzte Figur geschlagen wurde oder ein Bauer vorgestoßen ist.
    * Die Anzahl an durchgeführten Zügen.
* Die gekürzte FEN-Schreibweise. Diese besteht im Gegensatz zur klassischen FEN-Schreibweise nur aus der Platzierung der Spielfiguren und der Farbe, die aktuell am Zug ist.
* Und ein Zwei-Tupel. Dieses Tupel besitzt als ersten Wert einen booleschen Wert, der die aktuelle Farbe darstellt, die am Zug ist. Der zweite Eintrag stellt eine ASCII-Repräsentation eines gesamten Schachbretts dar.

> TODO: Stroetmann hat nicht verstanden welche Parameter mitgegeben werden und welche zurückgegeben wird, muss gegeben sein
> TODO: Stroetmann hat den Zusammenhang zu Absatz vorher nicht verstanden

Die Funktion ``get_board_and_turn`` erstellt nun für eine mitgegebene ``fen`` die gekürzte FEN-Schreibweise und gibt diese zurück.

In [ ]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

> TODO: Darstellungsweise der Parameter und Rückgabewerte evtl. anpassen
> TODO: Begriff "situation" hat ihm im Verlauf der Arbeit nicht verstanden

Als Nächstes wird eine Funktion definiert, die die Suche innerhalb einer ``sequence`` ermöglicht. Diese ``sequence`` ist eine Liste bestehend aus Mengen. In diesen Mengen stehen die Spielsituationen in einer der drei zuvor erwähnten Darstellungsweisen. Bei der ``situation`` muss darauf geachtet werden, dass sie die gleiche Darstellung besitzt wie die Elemente in der ``sequence``. Sofern die Funktion ``find_situation_in_sequence`` die Spielsituation in einer der Mengen findet, wird der Mengenindex der Liste zurückgegeben. Andernfalls lautet der Rückgabewert -1.

In [ ]:
def find_situation_in_sequence(situation, sequence):
    for i in range(len(sequence)):
        if situation in sequence[i]:
            return i
    return -1

Im weiteren Verlauf dieser Studienarbeit werden in dem Notebook ``02_calculation.ipynb`` Ergebnisse berechnet, die für eine spätere Validierung benötigt werden. Da die Ergebnisse eine beachtliche Größe (genaue Zahl) besitzen, werden diese lokal abgespeichert und müssen für eine spätere Verwendung erneut geladen werden. Hierfür wurde die Funktion ``load_s_n_fens`` definiert. Diese nimmt den Dateinamen (``filename``) einer ``.chessTest`` entgegen, die in dem Ordner ``S_n_Results`` gespeichert werden. Aus dieser Datei werden zwei Listen von Mengen generiert, die Spielsituation in sowohl der FEN-Schreibweise (``s_n_sequence_fen``) und der gekürzten FEN-Schreibweise(``s_n_sequence_short_fen``) beinhalten.

In [ ]:
def load_s_n_fens(filename):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    with (open("S_n_Results/" + filename + ".chessTest", "rb")) as test_fens:
        tmp = pickle.loads(test_fens.read())
        for item in tmp:
            tmp_set_fen = set()
            tmp_set_short = set()
            for fen in item:
                tmp_set_fen.add(fen)
                tmp_set_short.add(get_board_and_turn(fen))
            s_n_sequence_fen.append(tmp_set_fen)
            s_n_sequence_short_fen.append(tmp_set_short)
    return s_n_sequence_fen, s_n_sequence_short_fen

> TODO: Konzept mit dem Integer beschreiben - ausreichend?

## Effiziente Speichernutzung
Um den Arbeitsspeicher effizient zu nutzen, können Spielsituationen nicht als Objekte der ``chess`` Bibliothek gespeichert werden.
Einer ausgabe von `sys.getsizeof` zufolge benötigt ein Board-Objekt 48 Byte Speicher. Das ist bereits doppelt so viel, wie die 24 Byte eines Standard-Integers.
`sys.getsizeof` bestimmt jedoch nur die Größe des eigentlichen Objektes. Verwaltet dieses Pointer auf andere Objekte, ist die tatsächliche Größe höher.
Um nicht pro Stellung ein Objekt mit >=48 Byte zu benötigen werden die Positionen der Figuren als Binärer-Wert kodiert und als Integer gespeichert. 

Die Funktion `to_integer` berechnet für ein `board` und eine Liste der darauf befindlichen Figuren `piece_list` diese Zahl.
Es ist wichtig die Liste der Figuren als Übergabeparameter zu erhalten und nicht zu berechnen, da die Position einer Figur in der Liste mit der Position im Binärcode übereinstimmt.
Bei der Dekodierung wird aus dieser Position die Figur bestimmt. 
Das geringwertigste Bit speichert die Information, welcher Spieler am Zug ist.
Die folgenden (pro Figur 6) Bits kodieren die Zahl, welche der Position einer Figur auf dem Spielfeld entspricht.
Die Reihenfolge der Figuren entspricht deren Reihenfolge in der `piece_list`.
  
Die Funktion gibt einen Integer mit 24 oder 32 Byte Größe (32 Byte bei Bauern in der Stellung) zurück, welcher die Spielsituation repräsentiert.

In [ ]:
def to_integer(board, piece_list):
    tmp_list = piece_list.copy()
    piece_map = board.piece_map()
    int_rep = 1 if board.turn else 0
    
    pawn = chess.Piece.from_symbol("P")
    if pawn in piece_map.values():
        tmp_list.append(pawn)
        # Pawn and Queen will not be on the same board
        # This is in line with the tasks in readme.md
        piece_map[127] = chess.Piece.from_symbol('Q')
        
    for piece in piece_map:
        index = tmp_list.index(piece_map[piece])
        tmp_list[index] = None
        int_rep |= piece << index * 7 + 1 
        
    return int_rep

Die Funktion `to_board` wandelt einen Integer und eine Liste von Figuren `piece_list` in einer Spielsituation in ein Board-Objekt.
Das geringwertigste Bit, wird in die Information welcher Spieler am Zug ist konvertiert.
Die verbleibenden Bits werden nacheinander zu einer Zahl gewandelt, welche einem Spielfeld (`square`) entspricht. 

Die bestimmten Informationen über Zug und Piece-Map werden einem neuen Board-Objekt hinzugefügt, welches von der Funktion zurückgegeben wird.

In [ ]:
def to_board(int_rep, piece_list):
    tmp_list = piece_list.copy()
    pawn = chess.Piece.from_symbol("P")         
    tmp_list.append(pawn)
    
    n = len(piece_list)
    i = 0
    new_piece_map = {}

    turn = int_rep & 1
    int_rep = int_rep >> 1

    while int_rep != 0:
        pos = (int_rep & 127 << i * 7) >> i * 7
        if pos > 63:
            i += 1
            continue
        new_piece_map[pos] = tmp_list[i]
        i += 1
        int_rep = int_rep & ((2**(n*8)-1) << i * 7)

    res = chess.Board(None)
    res.turn = bool(turn)
    res.set_piece_map(new_piece_map)
    return res


> TODO: Kommentare mit Parameter und Rückgabewerte schreiben
> TODO: Konzept mit dem Integer beschreiben

In [ ]:
def to_integer(board, piece_list):
    piece_map = board.piece_map()

    # @Lukas why def of turn_int?
    turn_int = 1 if board.turn else 0
    int_rep = turn_int
    
    for piece in piece_map:
        index = piece_list.index(piece_map[piece])
        int_rep |= piece << index * 7 + 1 
        
    return int_rep

> TODO: Kommentare mit Parameter und Rückgabewerte schreiben

In [ ]:
def to_board(int_rep, piece_list):
    n = len(piece_list)
    i = 0
    new_piece_map = {}

    turn = int_rep & 1
    int_rep = int_rep >> 1

    while int_rep != 0:
        pos = (int_rep & 127 << i * 7) >> i * 7
        new_piece_map[pos] = pieces[i]
        i += 1
        int_rep = int_rep & ((2**(n*8)-1) << i * 7)
    res = chess.Board(None)
    res.turn = bool(turn)
    res.set_piece_map(new_piece_map)
    return res
